Setup and import libraries

In [ ]:
import os
import pandas as pd
import numpy as np

# Paths of the files to be used for formatting. (Specify with extension)
IMPORT_FILE = r'D:\IMP_2022.csv'
EXPORT_FILE = r'D:\EXP_2022.csv'

# Folder where the files will be saved.
EXPORT_DIRECTORY = r'C:\Users\Eckesaht\Desktop\as'

Extract

In [ ]:
# Reading the data from CSV files
imp = pd.read_csv(IMPORT_FILE, sep=';')
exp = pd.read_csv(EXPORT_FILE, sep=';')

Transform

In [ ]:
# Iterate over each unique state code
for uf in UFS:
    # Filter data for the current state
    filtered_uf_imp = imp[imp['SG_UF_NCM'] == uf]
    filtered_uf_exp = exp[exp['SG_UF_NCM'] == uf]

    # Pivot tables for import and export data
    filtered_uf_imp = filtered_uf_imp.pivot_table(index=['NCM', 'SG_UF_NCM'], columns='CO_MES', values='VL_FOB', aggfunc='sum').reset_index()
    filtered_uf_exp = filtered_uf_exp.pivot_table(index=['NCM', 'SG_UF_NCM'], columns='CO_MES', values='VL_FOB', aggfunc='sum').reset_index()

    # Reorder columns
    specified_order_imp = ['NCM', 'Imp_jan', 'Imp_fev', 'Imp_mar', 'Imp_abr', 'Imp_mai', 'Imp_jun', 'Imp_jul', 'Imp_ago', 'Imp_set', 'Imp_out', 'Imp_nov', 'Imp_dez']
    specified_order_exp = ['NCM', 'Exp_jan', 'Exp_fev', 'Exp_mar', 'Exp_abr', 'Exp_mai', 'Exp_jun', 'Exp_jul', 'Exp_ago', 'Exp_set', 'Exp_out', 'Exp_nov', 'Exp_dez']
    filtered_uf_imp = filtered_uf_imp.reindex(columns=specified_order_imp)
    filtered_uf_exp = filtered_uf_exp.reindex(columns=specified_order_exp)

    # Merging import and export data
    merged_sheet = pd.merge(filtered_uf_imp, filtered_uf_exp, on=['NCM'], how='outer')

    # Organizing columns in the correct order
    correct_order = ['NCM', 'Exp_jan', 'Imp_jan', 'Net_jan', 'Exp_fev', 'Imp_fev', 'Net_fev', 'Exp_mar', 'Imp_mar', 'Net_mar', 'Exp_abr', 'Imp_abr', 'Net_abr', 'Exp_mai', 'Imp_mai', 'Net_mai', 'Exp_jun', 'Imp_jun', 'Net_jun', 'Exp_jul', 'Imp_jul', 'Net_jul', 'Exp_ago', 'Imp_ago', 'Net_ago', 'Exp_set', 'Imp_set', 'Net_set', 'Exp_out', 'Imp_out', 'Net_out', 'Exp_nov', 'Imp_nov', 'Net_nov', 'Exp_dez', 'Imp_dez', 'Net_dez', 'Exp_2022', 'Imp_2022', 'Net_2022']
    merged_sheet = merged_sheet.reindex(columns=correct_order)

    # Calculating net values
    for month in ['jan', 'fev', 'mar', 'abr', 'mai', 'jun', 'jul', 'ago', 'set', 'out', 'nov', 'dez']:
        net_col = f'Net_{month}'
        exp_col = f'Exp_{month}'
        imp_col = f'Imp_{month}'

        # Fill NaN values with zeros for calculation
        merged_sheet[exp_col] = merged_sheet[exp_col].fillna(0)
        merged_sheet[imp_col] = merged_sheet[imp_col].fillna(0)

        # Calculate net values
        merged_sheet[net_col] = merged_sheet[exp_col] - merged_sheet[imp_col]

    # Summing up yearly values
    merged_sheet['Exp_2022'] = merged_sheet.filter(like='Exp_').sum(axis=1)
    merged_sheet['Imp_2022'] = merged_sheet.filter(like='Imp_').sum(axis=1)
    merged_sheet['Net_2022'] = merged_sheet.filter(like='Net_').sum(axis=1)

Load

In [ ]:
# Exporting the merged data to a CSV file
export_path = os.path.join(EXPORT_DIRECTORY, f'{uf}.csv')
merged_sheet.to_csv(export_path, index=False, sep=',')